In [32]:
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
from src.learn_data_struct import DataSet

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [33]:
class R2HandRilNet(torch.nn.Module):
    def __init__(self):
        super(R2HandRilNet, self).__init__()

        self.conv1 = torch.nn.Conv2d(
            in_channels=1, out_channels=1, kernel_size=3, stride=1, padding=1)
        self.act1 = torch.nn.ReLU()
        self.conv2 = torch.nn.Conv2d(
            in_channels=1,out_channels=1, kernel_size=3, stride=1, padding=1)
        self.act2 = torch.nn.ReLU()
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.act3 = torch.nn.Tanh()

        self.conv3 = torch.nn.Conv2d(
            in_channels=1, out_channels=1, kernel_size=3, stride=1, padding=1)
        self.act4 = torch.nn.ReLU()
        self.conv4 = torch.nn.Conv2d(
            in_channels=1,out_channels=1, kernel_size=3, stride=1, padding=1)
        self.act5 = torch.nn.ReLU()
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.act6 = torch.nn.Tanh()

        self.conv5 = torch.nn.Conv2d(
            in_channels=1, out_channels=1, kernel_size=3, stride=1, padding=1)
        self.act7 = torch.nn.ReLU()
        self.conv6 = torch.nn.Conv2d(
            in_channels=1,out_channels=1, kernel_size=3, stride=1, padding=1)
        self.act8 = torch.nn.ReLU()
        self.pool3 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.act9 = torch.nn.Tanh()

        self.conv7 = torch.nn.Conv2d(
            in_channels=1, out_channels=1, kernel_size=3, stride=1, padding=1)
        self.act10 = torch.nn.ReLU()
        self.conv8 = torch.nn.Conv2d(
            in_channels=1,out_channels=2, kernel_size=3, stride=1, padding=1)
        self.act11 = torch.nn.ReLU()
        self.pool4 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.act12 = torch.nn.Tanh()

        self.conv9 = torch.nn.Conv2d(
            in_channels=1, out_channels=1, kernel_size=3, stride=1, padding=1)
        self.act13 = torch.nn.ReLU()
        self.conv10 = torch.nn.Conv2d(
            in_channels=1,out_channels=1, kernel_size=3, stride=1, padding=1)
        self.act14 = torch.nn.ReLU()
        self.pool5 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.act14 = torch.nn.Tanh()

        self.conv11 = torch.nn.Conv2d(
            in_channels=1, out_channels=8, kernel_size=1, stride=1, padding=0)
        self.act15 = torch.nn.ReLU()

        self.fc1 = torch.nn.Linear(8, 63)
        self.act16 = torch.nn.ReLU()

        self.fc2 = torch.nn.Linear(63, 63)
        self.act17 = torch.nn.Softmax()
  
    def forward(self, x):

        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.pool1(x)
        x = self.act3(x)

        x = self.conv3(x)
        x = self.act4(x)
        x = self.conv4(x)
        x = self.act5(x)
        x = self.pool2(x)
        x = self.act6(x)

        x = self.conv5(x)
        x = self.act7(x)
        x = self.conv6(x)
        x = self.act8(x)
        x = self.pool3(x)
        x = self.act9(x)

        x = self.conv7(x)
        x = self.act10(x)
        x = self.conv8(x)
        x = self.act11(x)
        x = self.pool4(x)
        x = self.act12(x)

        x = self.conv9(x)
        x = self.act13(x)
        x = self.conv10(x)
        x = self.act13(x)
        x = self.pool5(x)
        x = self.act14(x)

        x = self.conv11(x)
        x = self.act15(x)

        x = self.fc1(x)
        x = self.act16(x)

        x = self.fc2(x)
        x = self.act17(x)

        return x

handrilnet = R2HandRilNet() 

In [34]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
handrilnet = handrilnet.to(device)

In [35]:
loss = torch.nn.CrossEntropyLoss()   #variable parameter
optimizer = torch.optim.Adam(handrilnet.parameters(), lr=1.0e-3)

In [37]:
train_dataset = DataSet(62)
test_dataset = DataSet(1)
x_train = train_dataset.batch().data
y_train = train_dataset.batch().letter
x_test = test_dataset.batch().data
y_test = test_dataset.batch().letter

FileNotFoundError: [Errno 2] No such file or directory: '../cut_img/5'

In [ ]:
#x_train = x_train.unsqueeze(1).float()
#x_test = x_test.unsqueeze(1).float()

In [ ]:
#x_train.shape()

In [9]:
batch_size = 10

test_accuracy_history = []
test_loss_history = []

x_test = x_test.to(device)
y_test = y_test.to(device)

for epoch in range(5000):
    order = np.random.permutation(len(x_train))
    for start_index in range(0, len(x_train), batch_size):
        optimizer.zero_grad()

        batch_indexes = order[start_index:start_index+batch_size]

        x_batch = x_train[batch_indexes].to(device)
        y_batch = y_train[batch_indexes].to(device)

        preds = handrilnet.forward(x_batch)

        loss_value = loss(preds, y_batch)
        loss_value.backward()

        optimizer.step()

    test_preds = handrilnet.forward(x_test)
    test_loss_history.append(loss(test_preds, y_test).to(device))

    accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().to(device)
    test_accuracy_history.append(accuracy)

    print(accuracy)

NameError: name 'x_test' is not defined

In [ ]:
handrilnet.forward(x_test)

In [ ]:
plt.plot(test_accuracy_history)
# plt.plot(test_loss_history)